In [2]:
#import libraries
import base64
import requests #pip install requests
import datetime
from urllib.parse import urlencode

In [3]:
#setting up variables
client_id = '5aedd1d0682f46618896dcea6108e3b4'
client_secret = 'e3c2c283017d452a93ddc951053196d2'
access_token = None

In [4]:
class ReadArtists():
    artist_list = None
    
    #reading all the artist from this list. all artist names will be held here
    def getList():
        file = open("artists.txt","r")
        raw_list = file.read()
        artist_list = raw_list.split(";")
        file.close()
        return artist_list
    
   # def writeList(filename):
    #    with open(filename, 'w') as f:
            
    
    

In [5]:
#https://www.youtube.com/watch?v=xdq6Gz33khQ&t=1318s
#code derived from link above
class SpotifyAPI():
    access_token = None
    access_token_expires = None
    access_token_hasExpired = True
    c_id = None
    c_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret

    def encode_credentials (self):
        id = self.client_id
        secret = self.client_secret
        if secret == None or id == None:
            raise Exception("client id or secret is not set")
        client_creds = f"{id}:{secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def token_headers(self):
        c64_credentials = self.encode_credentials()
        return {
            "Authorization": f"Basic {c64_credentials}"
        }
    
    def token_data(self):
        return {
            "grant_type": "client_credentials"
        }
    
    def get_auth(self):
        r= requests.post(self.token_url, data=self.token_data(), headers=self.token_headers())
        if r.status_code in range(200,299):
            now = datetime.datetime.now()
            self.access_token = r.json()['access_token']
            expires = now + datetime.timedelta(seconds=r.json()['expires_in'])
            self.access_token_expires = expires
            self.access_token_hasExpired = expires < now
            print("success")
        print(r.status_code)

In [6]:
class SpotifyQuery():
    headers = None
    artist_lookup = None
    endpoint_url = "https://api.spotify.com/v1/"
    endpoint_type = None
    enpoint_limit = None
    endpoint = None
    offset_count = 0
    artist_row = []
    
    def __init__(self, access_token):
        self.headers = {
                "Authorization": f"Bearer {access_token}"
        }
        
    def get_request(self, endpoint_query):
        self.endpoint = self.endpoint_url+self.endpoint_type+endpoint_query
        #print(self.endpoint)
        return requests.get(self.endpoint, headers = self.headers)
    
    
    def get_artistid(self):
        artist_list = ReadArtists.getList()
        self.endpoint_type = "search?"
        self.endpoint_limit = 1
        for i in artist_list:
            col = []
            artist_lookup = i
            endpoint_query = urlencode({"q": artist_lookup,"type": "artist", "limit": self.endpoint_limit})
            #self.endpoint = self.endpoint_url+self.endpoint_type+endpoint_query
            #r = requests.get(self.endpoint, headers = self.headers)
            r = self.get_request(endpoint_query)
            #print(r.status_code)
            items = r.json()['artists']['items'][0]
            col.append(items['name'])
            col.append(items['id'])
            self.artist_row.append(col)
            #print(items)\
            
    def calc_off(self):
        if self.offset_count > 0:
            offset = self.endpoint_limit*self.offset_count-1
        else:
            offset = 0
        self.offset_count += 1
        return offset 
    
    def get_song_data(self, artist_list, isAlbum):
        offset = self.calc_off()
        if isAlbum:
            endpoint_query = urlencode({"include_groups": "single,appears_on", "limit": self.endpoint_limit, "offset": offset})
        else:
            endpoint_query = urlencode({"limit": self.endpoint_limit, "offset": offset})
        r = self.get_request(endpoint_query)
        #print(r.status_code)
        items = r.json()['items']
        for item in items:
            list_col = []
            artists = item['artists']
            if len(artists) > 1:
                list_col.append(item['name'])
                list_col.append(item['id'])
                for artist in artists:  
                    list_col.append(artist['name'])
                    list_col.append(artist['id'])
                artist_list.append(list_col)
        if len(items) == 50:
            #print("offset: "+ str(self.offset_count) + " items: " + str(len(items)))
            self.get_song_data(artist_list, True)
    
    def get_albums_data(self, album_list):
        offset = self.calc_off()
        endpoint_query = urlencode({"limit": self.endpoint_limit, "offset": offset})
        r = self.get_request(endpoint_query)
        #print(r.status_code)
        items = r.json()['items']
        for item in items:
            list_col = []
            artists = item['artists']
            if item['album_group'] == "album":
                list_col.append(item['name'])
                list_col.append(item['id'])
                album_list.append(list_col) 
        if len(items) == 50:
            #print("offset: "+ str(self.offset_count) + " items: " + str(len(items)))
            self.get_albums_data(album_list)
    
    def get_albums(self):
        self.endpoint_limit = 50
        artist_list = []
        album_list = []
        for i in self.artist_row:
            self.offset_count = 0
            artist_id = i[1]
            self.endpoint_type = "artists/"+artist_id+"/albums?"
            self.get_song_data(artist_list, True)
            self.offset_count = 0
            self.get_albums_data(album_list)    
        lists_ret = dict()
        lists_ret['artists_link'] = artist_list
        lists_ret['all_albums'] = album_list
        return(lists_ret)
    
    def get_songs(self, lists_ret):
        self.endpoint_limit = 50
        artist_list = lists_ret['artists_link']
        album_list = lists_ret['all_albums']
        for album in album_list:
            self.offset_count = 0
            album_id = album[1]
            self.endpoint_type = "albums/"+album_id+"/tracks?"
            self.get_song_data(artist_list, False)
        return artist_list
    

In [7]:
spotify = SpotifyAPI(client_id, client_secret)
spotify.get_auth()
access_token = spotify.access_token
query = SpotifyQuery(access_token)
query.get_artistid()
print(query.artist_row)
dictionary = query.get_albums()

success
200
[['Drake', '3TVXtAsR1Inumwj472S9r4'], ['Bad Bunny', '4q3ewBCX7sLwd24euuV69X']]


In [9]:
songList = query.get_songs(dictionary)
cleanSongList = [];
found = False

for i in range(0, len(songList)):
    for j in range(0, len(cleanSongList)):
        if songList[i][0] == cleanSongList[j][0]:
            found = True
            break
    if not found:
        cleanSongList.append(songList[i])
    found = False
for song in cleanSongList:
    print(song)
    print("\n")

['STAYING ALIVE (feat. Drake & Lil Baby)', '0oEsQwXJhNbbb8VT0xOMok', 'DJ Khaled', '0QHgL1lAIqAw0HtD7YldmP', 'Drake', '3TVXtAsR1Inumwj472S9r4', 'Lil Baby', '5f7VJjfbwm532GiveGC0ZK']


['Over The Top (feat. Drake)', '4ROMS3rcR9JepxAtDRpvc5', 'Smiley', '6jeg7JBX9J9097esK752iR', 'Drake', '3TVXtAsR1Inumwj472S9r4']


['Seeing Green', '4o2xz4a9BjYCHh2xfYH50t', 'Nicki Minaj', '0hCNtLu0JehylgoiP8L4Gh', 'Drake', '3TVXtAsR1Inumwj472S9r4', 'Lil Wayne', '55Aa2cqylxrFIXC767Z865']


['B.B. King Freestyle', '4Epq84TTM5hHH5jI78R271', 'Lil Wayne', '55Aa2cqylxrFIXC767Z865', 'Drake', '3TVXtAsR1Inumwj472S9r4']


['You’re Mines Still (feat. Drake)', '7MQJy9di8JoLJOkG5mZAry', 'BLEU', '3KNIG74xSTc3dj0TRy7pGX', 'Drake', '3TVXtAsR1Inumwj472S9r4']


['Laugh Now Cry Later (feat. Lil Durk)', '0qGdc7fNq9RNIPEzZufa43', 'Drake', '3TVXtAsR1Inumwj472S9r4', 'Lil Durk', '3hcs9uc56yIGFCSy9leWe7']


['TWIST & TURN (feat. Drake & PARTYNEXTDOOR)', '2Yl3e3mavQghu4a6LmW79H', 'Popcaan', '62DmErcU7dqZbJaDqwsqzR', 'Drake', '3TVXt

In [17]:
import csv

#create edge list

edgelist = [["Target", "Source"]]
#Assumes max of 3 artists per song
for song in cleanSongList:
    size = len(song)
    if size == 4:
        continue
    elif size == 6:
        edgelist.append([song[3],song[5]])
    elif size == 8:
        edgelist.append([song[3],song[5]])
        edgelist.append([song[3],song[7]])
        edgelist.append([song[5],song[7]])

with open('edgelist.csv', 'w') as f:
    write = csv.writer(f)
    write.writerows(edgelist)


# with open('songlist.csv', 'w') as f:
#     write = csv.writer(f)
#     write.writerows(cleanSongList)